In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
filepath = "C:/Users/gufra/OneDrive/Desktop/Detection-TonIoT23-main/UNSW_2018_IoT_Botnet_Full5pc_4.csv"

In [3]:
training_dataset = pd.read_csv(filepath,nrows=600000)

In [4]:
training_dataset = training_dataset.drop(['stime'], axis=1)
training_dataset = training_dataset.drop(['flgs'], axis=1)
training_dataset = training_dataset.drop(['saddr'], axis=1)
training_dataset = training_dataset.drop(['daddr'], axis=1)
training_dataset = training_dataset.drop(['ltime'], axis=1)
training_dataset = training_dataset.drop(['state'], axis=1)
training_dataset
#replacing values
training_dataset['proto'] = training_dataset['proto'].replace(
    {'udp': '0', 'tcp': '1', 'icmp': '2', 'arp': '3', 'ipv6-icmp': '4'})
training_dataset['category'] = training_dataset['category'].replace(
    {'DDoS': '0', 'Normal': '1', 'Reconnaissance': '2', 'Theft': '3'})
training_dataset['subcategory'] = training_dataset['subcategory'].replace(
    {'UDP': '0', 'Normal': '1', 'OS_Fingerprint': '2', 'Service_Scan': '3', 'Data_Exfiltration': '4',
     'Keylogging': '5'})


In [14]:
training_dataset

,pkSeqID,flgs_number,proto,proto_number,sport,dport,pkts,bytes,state_number,seq,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,3000001,1,0,3,6226,80,15,900,4,109223,...,1.09825,100,100,1.09827,1.09825,1500,1500,1,0,0
1,3000002,1,0,3,6227,80,15,900,4,109224,...,1.09825,100,100,1.09827,1.09825,1500,1500,1,0,0
2,3000003,1,0,3,6228,80,15,900,4,109225,...,1.09825,100,100,1.09827,1.09825,1500,1500,1,0,0
3,3000004,1,0,3,6229,80,15,900,4,109226,...,1.09825,100,100,1.09827,1.09825,1500,1500,1,0,0
4,3000005,1,0,3,6230,80,15,900,4,109227,...,1.09825,100,100,1.09827,1.09825,1500,1500,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,3599996,1,0,3,39704,31625,1,60,4,4780,...,20788.50000,58,14,27874.60000,0.00000,58,14,1,2,3
599996,3599997,1,0,3,38759,31195,1,60,4,4781,...,20788.50000,58,14,37383.20000,0.00000,58,14,1,2,3
599997,3599998,6,0,3,35008,3401,1,60,4,4782,...,20788.50000,58,15,24193.50000,16949.20000,58,15,1,2,3
599998,3599999,1,2,4,771,18701,1,70,5,4783,...,0.00000,12,42,0.00000,0.00000,12,42,1,2,3


In [5]:
#handling the infinite value
training_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
training_dataset.fillna(training_dataset.mean(), inplace=True)
training_dataset


,pkSeqID,flgs_number,proto,proto_number,sport,dport,pkts,bytes,state_number,seq,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,3000001,1,0,3,6226,80,15,900,4,109223,...,1.09825,100,100,1.09827,1.09825,1500,1500,1,0,0
1,3000002,1,0,3,6227,80,15,900,4,109224,...,1.09825,100,100,1.09827,1.09825,1500,1500,1,0,0
2,3000003,1,0,3,6228,80,15,900,4,109225,...,1.09825,100,100,1.09827,1.09825,1500,1500,1,0,0
3,3000004,1,0,3,6229,80,15,900,4,109226,...,1.09825,100,100,1.09827,1.09825,1500,1500,1,0,0
4,3000005,1,0,3,6230,80,15,900,4,109227,...,1.09825,100,100,1.09827,1.09825,1500,1500,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,3599996,1,0,3,39704,31625,1,60,4,4780,...,20788.50000,58,14,27874.60000,0.00000,58,14,1,2,3
599996,3599997,1,0,3,38759,31195,1,60,4,4781,...,20788.50000,58,14,37383.20000,0.00000,58,14,1,2,3
599997,3599998,6,0,3,35008,3401,1,60,4,4782,...,20788.50000,58,15,24193.50000,16949.20000,58,15,1,2,3
599998,3599999,1,2,4,771,18701,1,70,5,4783,...,0.00000,12,42,0.00000,0.00000,12,42,1,2,3


In [6]:
y = training_dataset['attack']
x = training_dataset.drop(['attack'], axis=1)
x

,pkSeqID,flgs_number,proto,proto_number,sport,dport,pkts,bytes,state_number,seq,...,AR_P_Proto_P_SrcIP,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,category,subcategory
0,3000001,1,0,3,6226,80,15,900,4,109223,...,1.09825,1.09825,100,100,1.09827,1.09825,1500,1500,0,0
1,3000002,1,0,3,6227,80,15,900,4,109224,...,1.09825,1.09825,100,100,1.09827,1.09825,1500,1500,0,0
2,3000003,1,0,3,6228,80,15,900,4,109225,...,1.09825,1.09825,100,100,1.09827,1.09825,1500,1500,0,0
3,3000004,1,0,3,6229,80,15,900,4,109226,...,1.09825,1.09825,100,100,1.09827,1.09825,1500,1500,0,0
4,3000005,1,0,3,6230,80,15,900,4,109227,...,1.09825,1.09825,100,100,1.09827,1.09825,1500,1500,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,3599996,1,0,3,39704,31625,1,60,4,4780,...,19337.00000,20788.50000,58,14,27874.60000,0.00000,58,14,2,3
599996,3599997,1,0,3,38759,31195,1,60,4,4781,...,23102.30000,20788.50000,58,14,37383.20000,0.00000,58,14,2,3
599997,3599998,6,0,3,35008,3401,1,60,4,4782,...,18703.20000,20788.50000,58,15,24193.50000,16949.20000,58,15,2,3
599998,3599999,1,2,4,771,18701,1,70,5,4783,...,0.00000,0.00000,12,42,0.00000,0.00000,12,42,2,3


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, random_state=10, test_size=0.3)

In [8]:
Y_train

428558    1
321006    1
29924     1
385585    1
522836    1
         ..
299741    1
533660    1
105595    1
443712    1
345353    1
Name: attack, Length: 420000, dtype: int64

In [9]:
model = SVC()
start = time.time()
print('program start...')
print()
model.fit(X_train, Y_train)
print()
print(model.score(X_test, Y_test))
print()

y_pred = model.predict(X_test)
print(y_pred)
print()

end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

program start...


0.9993277777777778

[1 1 1 ... 1 1 1]

program end...

time cost: 
163.385183095932 seconds


In [10]:
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))

Classifiction Report :
              precision    recall  f1-score   support

           0       1.00      0.23      0.38       158
           1       1.00      1.00      1.00    179842

    accuracy                           1.00    180000
   macro avg       1.00      0.62      0.69    180000
weighted avg       1.00      1.00      1.00    180000

